<!-- This is used for Markdown rendering -->
<script
  src="https://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML"
  type="text/javascript">
</script>
<small>Base article: [PySDR](https://pysdr.org/)</small>

# Fourier transform [art.](https://pysdr.org/content/frequency_domain.html#fourier-series)
## *Continuous:*
Time Domain -> Freq Domain: $X(f) = \int x(t) e^{-j2\pi ft} dt$<br/>
Freq Domain -> Time Domain: $x(t) = \frac{1}{2\pi} \int X(f)e^{j2\pi ft} df$

Angular Frequency: $w = 2\pi f$

## *Discrete:*
$$X_k = \sum^{N-1}_{n=0} x_n e^{-(\frac{j2π}{N})kn}$$

## Time-Frequency properties
1. Linear Property:<br/>
$ax(t) + by(t) \leftrightarrow aX(f) + bY(f)$

2. Frequency Shift Property:<br/>
If we take a signal x(t) and multiply it by a sine wave, in the frequency domain, we'll get X(f) but shifted by a frequency f0;<br/>
$e^{2\pi jf_0t}x(t) \leftrightarrow X(f-f_0)$

3. Scaling in Time Property
$x(at) \leftrightarrow X(\frac{f}{a})$<br/>
Scaling in time domain causes inverse scaling in the frequency domain. => The reason why higher data rate takes more bandwidth/spectrum.

4. Convolution In Time Property<br/>
$\int x(\tau) y(t - \tau) d\tau \leftrightarrow X(f)Y(f)$<br/>
Called convolution property because convolving x(t) & y(t) in time domain.

5. Convolution in Frequency Property<br/>
$x(t)y(t) \leftrightarrow \int X(g)Y(f-g)dg$


# Fast Fourier Transform (FFT) [art.](https://pysdr.org/content/frequency_domain.html#fast-fourier-transform-fft)
Algorithm to compute the DFT, most efficient at the moment. Take one input, has one output. Converts input signal from time to frequency domain.

Because output is in frequency domain, the span of the x-axis is based on the **sample rate**.

# IQ Sampling [art.](https://pysdr.org/content/sampling.html)
<small>a.k.a. Complex/Quadrature Sampling</small>

## Sampling basics
Sampling: Grabbing values from a function at regular moments in time.<br/>
E.g.: $S(t)$, random continuous function. We'll record the value of $S(t)$ at regular intervals of *T* seconds; the *sample period*. *Sample Rate* = $\frac{1}{T}$.

Mathematically, the sampling process is $S_n = S(nT)$ w/ n, an integer usually starting at 0.

## Nyquist Sampling
How fast must we sample ? With a sine function of frequency $f$ and a sample rate $Fs = f$. The reconstruction of this sampling will be a constant.
<img alt="Fs = f" src="https://pysdr.org/_images/sampling_Fs_0.3.svg" title="Fs = f emulates a constant function"/>
<img alt="Fs = 1.2 * f" src="https://pysdr.org/_images/sampling_Fs_0.36.svg" title="sampling of 1.2 * f" />
<img alt="Fs = 1.5 * f" src="https://pysdr.org/_images/sampling_Fs_0.45.svg" title="sampling of 1.5 * f" />

We need to sample at **twice** the frequency to remove any ambiguity. The minimum rate at which we can sample is called the *Nyquist Rate*.

When we don't sample fast enough, we get aliasing.

## Quadrature Sampling
Quadrature refers to waves that are 90 degrees out of phase.

Let's have:
$$Icos(2\pi ft)$$
$$Qsin(2\pi ft)$$

$cos()$ will ere be the "in phase" component. When transmitting an RF signal, the transmitter add the $sin()$ and the $cos()$:
$$x(t) = Icos(2\pi ft) + Qsin(2\pi ft)$$
Using the trig. identity:
$$x(t) = Acos(x - \phi)$$

When we add the two functions, we get another pure sine wave, with a diff. phase & ampl. This allows for an easier control of the resulting sine wave's phase & ampl, as we only have to change I & Q to create (approx.) any sine wave.

## Complex numbers
Complex Nu,bers have a magnitude and a phase (See the complex as a vector not a point).

For a given Complex $a+bj$,
$$\text{magnitude} = \sqrt{a^2 + b^2}$$
$$phase = tan^{-1}(\frac{b}{a})$$

In python,
```py
np.abs(x)  # Gives the magnitude
np.angle(x)  # Gives the phase
```

**Relation to IQ**: I is represented by the real part, Q by the imaginary.

## Complex Numbers in FFTs
FFTs output an array of complex numbers, each with phase & magnitude. This allows for the reconstruction of the original signal.

### Receiver Side
When a receiver (FM/...) receives a real signal, we transform this signal into *IQ* values. These are sampled individually and combined into a complex number $I + jQ$. E.g. With sampling rate: 2MHz sample rate means 2 million IQ samples per second.

In Practice, we do not need to manually separate the I & Q. The SDR does that for us. Same goes for the other way, we only need to provide I & Q samples for the transmission.

### Carrier & Downconversion
**Carrier**: Frequency of the sine wave actually sent through the air. It "carries" the information at a given frequency.

E.g.: Wifi, Bluetooth, ... Usually use carriers between 100MHz & 6GHz.

Changing the IQ values quickly and transmitting is call "Modulating" the carrier. By changing I & Q, we change the phase & amplitude of the carrier. An other option is to change the frequency, in FM radio for example.

Downconversion: Sampling a 2.4GHz signal would require a samplig rate of 4.8GHz, which is really expensive. Instead, we downconvert to signal, to sample it around DC, 0Hz.

Most signals are are 100kHz to 40MHz wide in bandwidth. We can then sample at a much lower rate (~80MHz at most).

The downconversion is performed by the SDR. We don't have to do anything else than telling it which frequency to tune to.

### Receiver Architecture
**Direct Conversion** (Zero IF): Downconvert the signal and split it into I & Q.<br/>
**Direct Sampling** (Direct RF): Sample fast enough to capture everything from 0Hz to 1/2 sample rate. => Requires Expensive ADC chip.<br/>
**Superheterodyne**: Downconversion but not all the way to zero. It places the signal at an intermediate frequency; IF. 

low noise amplifier (LNA): amplifier designed for extremely low power signals.

### Baseband & Bandpass Signals
Signal centered around 0Hz => "Baseband". "Bandpass"; signal existing at some RF frequency nowhere near 0, shifted up for the purpose of Wireless transmission.

Most of the time, IQ signals are at baseband frequency.

### DC Spike & Offset tuning
Often large spikes at the center of the FFT; "DC offset", "DC Spike" or "LO Leakage" (LO; Local Oscillator).Beceause SDR tunes to center frequency, 0Hz portion of FFT corresponds to this frequency => Spike.

### Calculating Average Power
W/ Discrete Complex Signal; Average power: $$P = \frac{1}{N}\sum^{N}_{n=1} |x[n]|^2$$

In python:
```py
avg_pwr = np.mean(np.abs(x)**2)
```

Trick: When the signal has roughly zero mean; signal's power can be found with the variance of the samples:
```py
avg_pwr = np.var(x)  # Signal should have roughly zero mean
```

=> Equation for the variance: $P = \frac{1}{N}\sum^{N}_{n=1} |x[n]-\mu|^2$. If $\mu \approx 0$, this is the same as the power.

### Calculating Power Spectral Density
We can convert a signal to the frequency domain w/ FFT => PSD (Power Spectral Density) but we need more than just the FFT;
* Take FFT of our samples => e.g. 1024 samples -> 1024 complex floats
* Take the magnitude of the FFT output => e.g. 1024 floats
* Normalize; Divide by FFT size => 1024 in this case
* Square resulting magnitude to get power
* Convert to dB w/ $10 log_{10}()$ (Always viewed in log scale)
* Perform FFT shift to move 0Hz to the center

In python:
```py
Fs = 1e6  # Sampled at 1MHz
N = 1024
x = x[:N]  # Only use the first 1024 samples
# Optionally, apply a window here
# x = x * np.hamming(len(x))
PSD = (np.abs(np.fft.ftt(x)) / N) ** 2
PSD_log = 10.0 * np.log10(PSD)
PSD_shifted = np.fft.fftshift(PSD_log)

# We can then shift the observation window to see around the frequency we tuned to
center_freq = 2.4e9  # Frequency we tuned our SDR to
f = np.arrange(Fs / -2.0, Fs / 2.0, Fs / N)  # Start, Stop, Step around 0Hz
f += center_freq  # Now add center frequency
plt.plot(f, PSD_shifted)
plt.show()
```



# Digital Modulation
## Symbols
Transmit signals will be made up of "Symbols". Each symbol will carry some number of bits of information & we will transmit signals back to back; thousands or millions in a row.

E.g. 8ns symbols with 4 symbols => 2 bits every 8ns; 250Mb/s; 4 cables for 1Gb/s<br/>
     8ns symbols with 16 symbols => 4 bits every 8ns; 500Mb/s

## Wireless symbols
Why can't we transmit ethernet signal through the air ?
1. Low frequencies require **huge** antennas
2. Square waves take an excessive amount of spectrum for the bits/s. sharp changes use a large amount of bandwidth.

Instead, for wireless signals, we start from a carrier, a sinusoid. We then modulate it in a certain way; what ways ?

First, what are the properties we can infer on ?
1. Amplitude
2. Phase
3. Frequency

### Amplitude Shift Keying (ASK)
Shift the amplitude to transmit the data; with a base amplitude of 1, we have A=1 means bit 0 & A=2 means bit 1. This is called *2-ASK*. We can use more than 2 levels (E.g. 4-ASK; each symbol carries 2 bits).

How to code that ? Create a vector with N samples per symbols, then multiply this vector by a sinusoid. This modulates the signal onto a carrier (the sinusoid).

### Phase Shift Keying (PSK)
Modulate the phase in a similar manner. Simplest form; Binary PSK; with two levels of phase: *No phase change* & *180 degree phase change*.

No easy to look at in time domain; we usually represent it in a complex plane;

### IQ Plot/Constellation
Representing IQ values on a complex plane shows the set of symbols that can be sent. With the example of BSPK, we'll have two dots, one in $1+0j$ & one in $-1+0j$.

For PSK, we always have N ≠ phases, equally spaced around 360 degrees. Often shown on the unit circle.

We can represent *ASK* on the same plane and see that 2-ASK = BSPK. We usually call it BSPK though.

### Quadrature Amplitude Modulation (QAM)
What if we combine ASK & PSK ? Looks like a grid on the IQ plane. It does not have to be a square (32-QAM can't build a square)

### Frequency Shift Keying (FSK)
Shift between ≠ sets of frequencies, where each one is a symbol. Because modulating over the carrier, the different frequencies are centered around our carrier frequency.

When using FSK, critical to ask what should the spacing btw freqs. be ? this spacing is usually denoted $\Delta f$ in Hz. We want to avoid overlap in frequency domain so it must be large enough.

## Differential coding
Do not transmit the data, transmit the changes.

```py
before_diff = [1, 1, 0, 0, 0, 1, 0]
after_diff = [1, 0, 1, 1, 1, 1, 0, 0]
```

Big downside: One bit error leads to two bits error.